In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

import cv2
import numpy as np

def get_background(file_path):
    cap = cv2.VideoCapture(file_path)
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Handle frames with different dimensions, if necessary
        # frame = cv2.resize(frame, (desired_width, desired_height))

        frames.append(frame)

    cap.release()  # Release the video capture object
    if frames:
        median_frame = np.median(frames, axis=0).astype(np.uint8)
        return median_frame
    else:
        # Handle the case when no frames are read
        print("Error: No frames were read from the video.")
        return None

# Example usage
background = get_background("HethenHisPelivideo.mp4")

if background is not None:
    print("Background data type:", background.dtype)
    background_gray = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
    print("Background data type after conversion:", background_gray.dtype)
else:
    print("Error: No valid background frame.")



def ORB_detector(new_image, image_template):
    orb = cv2.ORB_create(1000, 1.2)  # ORB detector of 1000 keypoints, scaling pyramid factor=1.2
    (kp1, des1) = orb.detectAndCompute(new_image, None)  # Detect keypoints on the new image
    (kp2, des2) = orb.detectAndCompute(image_template, None)  # Detect keypoints of the template image
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)  # Matcher
    matches = bf.match(des1, des2)  # Extract matches
    matches = sorted(matches, key=lambda val: val.distance)  # Sort matches
    img2 = cv2.drawKeypoints(image_template, kp2, None, color=(0,255,0), flags=0)
    img1 = cv2.drawKeypoints(new_image, kp2, None, color=(0,255,0), flags=0)
    plt.imshow(img2)
    plt.show()
    return len(matches)

# Load video file and template image

consecutive_frames = 4

background = get_background("He&HisPelivideo.mp4")
print("Background data type:", background.dtype)
background_gray = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)


frame_count = 0
frame_diff_list = []

while True:
    cap = cv2.VideoCapture("He&HisPelivideo.mp4")
    ret, frame = cap.read()

    frame_count += 1
    orig_frame = frame.copy()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY).astype(np.uint8)
    print("Background shape:", background.shape)
    print("Gray shape:", gray.shape)

    frame_diff = cv2.absdiff(gray, background_gray)
    ret, thres = cv2.threshold(frame_diff, 50, 255, cv2.THRESH_BINARY)
    dilate_frame = cv2.dilate(thres, None, iterations=2)
    frame_diff_list.append(dilate_frame)
    if len(frame_diff_list) == consecutive_frames:
        sum_frames = sum(frame_diff_list)
        contours, hierarchy = cv2.findContours(sum_frames, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            if cv2.contourArea(contour) < 500:
                continue
            (x, y, w, h) = cv2.boundingRect(contour)
            cv2.rectangle(orig_frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Get height and width of webcam frame
        height, width = frame.shape[:2]

        # Define ROI Box Dimensions (Note some of these things should be outside the loop)
        top_left_x = int(width / 3)
        top_left_y = int((height / 2) + (height / 4))
        bottom_right_x = int((width / 3) * 2)
        bottom_right_y = int((height / 2) - (height / 4))

        # Draw rectangular window for our region of interest
        # cv2.rectangle(frame, (top_left_x,top_left_y), (bottom_right_x,bottom_right_y), 255, 3)

        # Crop window of observation we defined above
        cropped = frame[bottom_right_y:top_left_y , top_left_x:bottom_right_x]

        # Flip frame orientation horizontally
        frame = cv2.flip(frame,1)

        # Get number of ORB matches 
        matches = ORB_detector(cropped, image_template)

        # Display status string showing the current no. of matches 
        output_string = "Matches = " + str(matches)
        cv2.putText(frame, output_string, (50,450), cv2.FONT_HERSHEY_COMPLEX, 2, (250,0,150), 2)

        # Our threshold to indicate object deteciton
        # For new images or lightening conditions you may need to experiment a bit 
        # Note: The ORB detector to get the top 1000 matches, 350 is essentially a min 35% match
        threshold = 336

        # If matches exceed our threshold then object has been detected
        if matches > threshold:
            cv2.rectangle(frame, (top_left_x,top_left_y), (bottom_right_x,bottom_right_y), (0,255,0), 3)
            cv2.putText(frame,'Object Found',(50,50), cv2.FONT_HERSHEY_COMPLEX, 2 ,(0,255,0), 2)
            # Display the frame with the detected object
        cv2.imshow('Object Detector using ORB', frame)

        cv2.imshow('Detected Objects', orig_frame)
        if cv2.waitKey(100) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()